<a href="https://colab.research.google.com/github/Varshinigarlapati963/Code_Tech_03/blob/main/CodeTech_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
# Required Libraries
import nltk
import numpy as np
import json
import random
import re
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.naive_bayes import MultinomialNB

# Download NLTK Data
nltk.download('punkt')
nltk.download('wordnet')

# Initialize Lemmatizer
lemmatizer = WordNetLemmatizer()

# Load intents
intents_data = {
    "intents": [
        {
            "tag": "greeting",
            "patterns": ["Hi", "Hello", "Hey", "Good morning", "Good evening", "Hi there", "What's up", "Yo", "Howdy"],
            "responses": ["Hello! 😊", "Hi there! 👋", "Hey! How can I help you?"]
        },
        {
            "tag": "name",
            "patterns": ["What is your name?", "Who are you?", "Tell me your name", "What's your name", "Your name?"],
            "responses": ["I'm CodTech Bot! 🤖", "I am your AI assistant."]
        },
        {
            "tag": "goodbye",
            "patterns": ["Bye", "Goodbye", "See you later", "See ya", "Catch you later", "Take care"],
            "responses": ["Goodbye! 👋", "Have a great day! 🌞", "See you again!"]
        },
        {
            "tag": "thanks",
            "patterns": ["Thanks", "Thank you", "Appreciate it", "Thx", "Thanks a lot", "Much appreciated"],
            "responses": ["You're welcome! 😊", "Anytime!", "Glad to help! 🙌"]
        },
        {
            "tag": "capabilities",
            "patterns": ["What can you do?", "What are your features?", "What do you know?", "How can you help me?"],
            "responses": [
                "I can answer questions, chat with you, and help you learn things! 📚",
                "I'm here to assist with small talk and basic questions.",
                "I can provide information, respond to greetings, and more!"
            ]
        },
        {
            "tag": "creator",
            "patterns": ["Who created you?", "Who made you?", "Your developer?", "Who built you?"],
            "responses": ["I was created by a developer just like you!", "An awesome human programmed me. 🤖"]
        },
        {
            "tag": "mood",
            "patterns": ["How are you?", "How's it going?", "Are you okay?", "You good?", "How do you feel?"],
            "responses": ["I'm doing great, thanks for asking! 😊", "Feeling code-tastic!", "Always ready to chat!"]
        }
    ]
}

# Save to intents.json (optional)
with open("intents.json", "w") as f:
    json.dump(intents_data, f, indent=4)

# Tokenization and Preprocessing
def clean_text(text):
    text = re.sub(r"[^\w\s]", "", text.lower())
    return word_tokenize(text)

all_words = []
tags = []
xy = []

for intent in intents_data['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        tokens = clean_text(pattern)
        tokens = [lemmatizer.lemmatize(w) for w in tokens]
        all_words.extend(tokens)
        xy.append((tokens, tag))

# Remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))

# Bag of Words
def bag_of_words(tokenized_sentence, all_words):
    tokenized_sentence = [lemmatizer.lemmatize(w.lower()) for w in tokenized_sentence]
    bag = np.zeros(len(all_words), dtype=np.float32)
    for idx, word in enumerate(all_words):
        if word in tokenized_sentence:
            bag[idx] = 1.0
    return bag

# Training data
X_train = []
y_train = []

for (pattern_sentence, tag) in xy:
    bow = bag_of_words(pattern_sentence, all_words)
    X_train.append(bow)
    y_train.append(tags.index(tag))

X_train = np.array(X_train)
y_train = np.array(y_train)

# Train the model
model = MultinomialNB()
model.fit(X_train, y_train)

# Prediction with confidence threshold
def predict_tag(sentence, threshold=0.3):
    sentence_tokens = clean_text(sentence)
    bow = bag_of_words(sentence_tokens, all_words).reshape(1, -1)
    probs = model.predict_proba(bow)[0]
    best_index = np.argmax(probs)
    if probs[best_index] >= threshold:
        return tags[best_index]
    return "unknown"

def get_response(tag):
    for intent in intents_data['intents']:
        if intent['tag'] == tag:
            return random.choice(intent['responses'])
    return random.choice([
        "Hmm... I'm not sure what you mean. 🤔",
        "I'm sorry, I didn't understand that. 😕",
        "Could you rephrase it?"
    ])

# Console Chat
def chat():
    print("CodTechBot: Hello! Type 'quit' to exit.")
    while True:
        msg = input("You: ")
        if msg.lower() == "quit":
            print("CodTechBot: Goodbye! 👋")
            break
        tag = predict_tag(msg)
        response = get_response(tag)
        print("Urs ChatBot:", response)

# Run
chat()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


CodTechBot: Hello! Type 'quit' to exit.
You: hi
Urs ChatBot: Hello! 😊
You: how are you
Urs ChatBot: I'm doing great, thanks for asking! 😊
You: what can you do
Urs ChatBot: I can provide information, respond to greetings, and more!
You: ur name?
Urs ChatBot: I'm CodTech Bot! 🤖
You: quit
CodTechBot: Goodbye! 👋
